In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f62f927f5e356fe3e9ca63c2ad544aae46b5ae6173a10125d51ce635ab4c43f0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-16 19:49:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.84MB/s    in 0.2s    

2022-12-16 19:49:38 (5.84 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://miral-bucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Jewelry_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...|2015-08-31 00:00:00|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...| 

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id" ,"star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [ ]:
print(vine_df.count())

1767753


In [ ]:
# total votes is greater than or equal to 20
total_votes = vine_df.filter("total_votes>=20")
total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [ ]:
helpful_votes = total_votes.filter("helpful_votes/total_votes>=0.5")
helpful_votes.show()
print(helpful_votes.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [ ]:
paid_vine_program = helpful_votes.filter("vine == 'Y'")
paid_vine_program.show()
print(paid_vine_program.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15PXA9XU9ZKSU|          4|           29|         29|   Y|                N|
|R2LJ4QYQ4PYDEH|          5|           24|         26|   Y|                N|
|R2PJWWQZ37WCIC|          5|           95|         96|   Y|                N|
|R33G2VDKRN2USY|          5|           21|         21|   Y|                N|
|R13R4X37T2U50A|          4|           52|         57|   Y|                N|
|R1X1J2NAQ39DF6|          4|           29|         34|   Y|                N|
|R2UV5XHFNMPXOX|          5|           60|         61|   Y|                N|
|R31V08BCC9X3MF|          5|           18|         22|   Y|                N|
|R2G00QWTYW9N2G|          5|           86|         91|   Y|                N|
| RL8H3NKKP989J|          5|           30|         34|   Y|     

In [ ]:
unpaid_vine_program = helpful_votes.filter("vine == 'N' ")
unpaid_vine_program.show()
print(unpaid_vine_program.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [ ]:

from pyspark.sql.functions import count,when
c = helpful_votes.groupBy("vine").agg(count("vine").alias("Total review"),count(when(helpful_votes["star_rating"] == 5, True)).alias("Total 5 star review"),((count(when(helpful_votes["star_rating"] == 5, True))/count("vine"))*100).alias("Percentage")).show()

+----+------------+-------------------+-----------------+
|vine|Total review|Total 5 star review|       Percentage|
+----+------------+-------------------+-----------------+
|   Y|          21|                 11|52.38095238095239|
|   N|        7689|               4444|57.79685264663805|
+----+------------+-------------------+-----------------+



In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for paid vine program

number_of_reviews = paid_vine_program.count()
number_of_5_star_reviews = paid_vine_program.filter("star_rating = 5").count()
percentage_of_5_star_review = (number_of_5_star_reviews/number_of_reviews) *100

print("Total number of reviews are: %f, The number of 5-star reviews are: %f and the percentage of 5-star reviews for paid vine program is: %f"
 %(number_of_reviews , number_of_5_star_reviews , percentage_of_5_star_review))

Total number of reviews are: 21.000000, The number of 5-star reviews are: 11.000000 and the percentage of 5-star reviews for paid vine program is: 52.380952


In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for unpaid vine program

number_of_reviews_up = unpaid_vine_program.count()
number_of_5_star_reviews_up = unpaid_vine_program.filter("star_rating = 5").count()
percentage_of_5_star_review_up = (number_of_5_star_reviews_up/number_of_reviews_up) *100

print("Total number of reviews are: %f , The number of 5-star reviews are: %f and the percentage of 5-star reviews for unpaid vine program is: %f"
 %(number_of_reviews_up , number_of_5_star_reviews_up , percentage_of_5_star_review_up))

Total number of reviews are: 7689.000000 , The number of 5-star reviews are: 4444.000000 and the percentage of 5-star reviews for unpaid vine program is: 57.796853
